In [ ]:
import time
import pickle
import numpy as np
import pandas as pd
# from model import model
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import FastFood as FF
from sklearn import svm, metrics, datasets, preprocessing
from sklearn.kernel_approximation import RBFSampler
from sklearn.model_selection import train_test_split
from sklearn import svm
# from cvxopt import normal
# from cvxopt.modeling import variable, op, max, sum
# import pylab
from scipy.optimize import minimize
import cvxpy as cp
from sklearn.metrics.pairwise import rbf_kernel
from pulp import *
from sklearn.preprocessing import StandardScaler  


In [ ]:
eta        = 1.
delta      = 0.001
h = 0.5


In [ ]:
"""Test model"""
def testModel(X, y, beta):
    ypred = np.dot(X, beta) 
    ypred[ypred >= 0] = 1
    ypred[ypred < 0] = -1
    pred_error = 1 - np.abs(metrics.accuracy_score(y.flatten(), ypred))
    return pred_error

In [ ]:
"""Non-private solution"""

def non_private(X, y, lambda2, T, epsilon, h):
    m, n = X.shape[0], X.shape[1]
    beta = np.zeros((n, 1)) 
    def huber_loss(w):
        m, n = X.shape[0], X.shape[1]        
        z = np.multiply(y, np.reshape(X.dot(w),(m,1)))
        obj = 0
        for i in range(m):
            if z[i,0] > (1+h):
                obj = obj
            elif np.abs(1 - z[i,0]) <= h:
                obj += (1 - z[i,0] + h)**2/(4*h) 
            elif z[i,0] < (1-h):
                obj += 1 - z[i,0]
        return obj/m + lambda2 * (np.linalg.norm(w) ** 2)/2
    w0 = np.ones(n)
    res = minimize(huber_loss, w0, method='Nelder-Mead', tol = 1e-09)
    beta = res['x']
    beta = np.reshape(np.array(beta), (n,1))
    return beta

In [ ]:
"""Noise generator"""
def noise(scale,Length):
    res = np.random.laplace(0, 1/scale, Length) #Generate the norm of noise according to gamma distribution
    return res

In [ ]:
"""Objective perturbation"""
def objective_pert(X, y, lambda2, T, epsilon, h):
    m, n = X.shape
    
    # huber loss
    c = 1/(2*h)

    epsilon2 = epsilon - np.log(1+2*c/(m*lambda2)+(c**2/((m*lambda2)**2)))
    if epsilon2 > 0:
        Delta = 0
    else:
        Delta = c / (m * (np.exp(epsilon/4)-1)) - lambda2
        epsilon2 = epsilon / 2
    b = noise(epsilon2/2,n)
    def huber_loss(w):
        m, n = X.shape[0], X.shape[1]        
        z = np.multiply(y, np.reshape(X.dot(w),(m,1)))
        obj = 0
        for i in range(m):
            if z[i,0] > (1+h):
                obj = obj
            elif np.abs(1 - z[i,0]) <= h:
                # huber loss
                obj += (1 + h - z[i,0])**2/(4*h)

            elif z[i,0] < (1-h):
                obj += 1 - z[i,0]
        return obj/m + lambda2 * (np.linalg.norm(w) ** 2)/2 + np.array(b).dot(w)/m + Delta*(np.linalg.norm(w) ** 2)/2
    w0 = np.ones(n)
    res = minimize(huber_loss, w0, method='Nelder-Mead', tol = 1e-09)
    beta = res['x']
    beta = np.reshape(np.array(beta), (n,1)) 
    return beta 

In [ ]:
"""Output perturbation"""
def output_pert(X, y, lambda2, T, epsilon, h):
    m, n = X.shape
    def huber_loss(w):
        m, n = X.shape[0], X.shape[1]        
        z = np.multiply(y, np.reshape(X.dot(w),(m,1)))
        obj = 0
        for i in range(m):
            if z[i,0] > (1+h):
                obj = obj
            elif np.abs(1 - z[i,0]) <= h:
                obj += (1 - z[i,0] + h)**2/(4*h) 
            elif z[i,0] < (1-h):
                obj += 1 - z[i,0]
        return obj/m + lambda2 * (np.linalg.norm(w) ** 2)/2
    w0 = np.ones(n)
    res = minimize(huber_loss, w0, method='Nelder-Mead', tol = 1e-09)
    beta = res['x']
    beta = np.reshape(np.array(beta), (n,1))
    b = noise((lambda2 * epsilon * m)/2,n)
    beta = np.reshape(beta, (n,1)) + np.reshape(b,(n,1))
    return beta

In [ ]:
"""Random Kitchen Sinks kernel approximation"""
def random_kitchen_sink(X, n):
    m, d = X.shape
    gamma = 1 / (d * X.var())
    W = np.random.randn(d, n)*np.sqrt(2 * gamma)
    b = np.random.uniform(-np.pi, np.pi, n)
    X_features = np.sqrt(2/n) * np.cos(X.dot(W) + b)
    return X_features

In [ ]:
def trainAggregateModel(X, y, modelName, methodName, Lambda, kernel, features, epsilons):
    switch = {
        "output_pert"          : output_pert,
        "objective_pert"       : objective_pert
    }
    X = approximation(X, kernel, features)
    normX = np.max(np.linalg.norm(X, axis=1))
    X = X/normX
    fun = switch.get(modelName)
    average_acc, betas_pri = [], []
    for epsilon in epsilons:
        errors = []
        for lambda2 in Lambda:
            error_cv = crossValidate(X, y, modelName, lambda2, T, epsilon, h, kernel, features)
            print('mean train error: ' + str(error_cv))
            errors.append(error_cv)
        p = np.argmin(errors)
        print(errors)
        #smallest error for each epsilon
        error_epsilon = errors[p] 
        # best lambda havinf smallest error
        lambda_opt = Lambda[p] 
        #get best beta using best lambda
        beta_opt = fun(X, y, lambda_opt, T, epsilon, h)      
        print("Relative Error is : " + str(error_epsilon) + ' when epsilon is ' + str(epsilon) + ' and lambda is ' + str(lambda_opt))
        betas_pri.append(beta_opt)
        average_acc.append(error_epsilon)
    plt.plot(average_acc)
    plt.show()
    return average_acc, beta_opt

In [ ]:
def trainNonPrivateModel(X, y, modelName, Lambda, kernel, features):

    errors_cv, errors_svm = [], []
    epsilon = 0
    X = approximation(X, kernel, features)
    for lambda2 in Lambda:
        errors_iteration_cv, errors_iteration_svm = [], []
        error_iteration_cv = crossValidate(X, y, modelName, lambda2, T, epsilon, h, kernel, features)
        print('mean train error: ' + str(error_iteration_cv))
        errors_cv.append(error_iteration_cv)

    clf = svm.SVC(gamma='scale')
    scores = cross_val_score(clf, X, y.flatten(), cv=10)
    errors_svm = np.mean([1 - score for score in scores])
    
    p_cv = np.argmin(errors_cv)
    
    error_epsilon_cv = errors_cv[p_cv] 
    
    lambda_opt_cv = Lambda[p_cv]
    
    print("Relative Error is : " + str(error_epsilon_cv) + ' when lambda is ' + str(lambda_opt_cv) + ' using optimal solver')
    print("Relative Error is : " + str(errors_svm) + ' using svc')
 
    beta_opt = non_private(X, y, lambda_opt_cv, T, epsilon, h)
    return error_epsilon_cv, errors_svm, beta_opt
    

In [ ]:
def crossValidate(X, y, modelName, lambda2, T, epsilon, h, kernel, features):
    switch = {
        "non_private"          : non_private,
        "output_pert"          : output_pert,
        "objective_pert"       : objective_pert,
    }
    fun = switch.get(modelName)
         
    kf = KFold(n_splits=10)
    errors_cv = []
    for train_index, test_index in kf.split(X):
        xtrain, ytrain = X[train_index], y[train_index]
        xtest, ytest = X[test_index], y[test_index]
        m, d = xtrain.shape[0], xtrain.shape[1]
        
        beta = fun(xtrain, ytrain, lambda2, T, epsilon, h)
        error = testModel(xtest, ytest, beta)
        print('train error: ' + str(error))
        errors_cv.append(error)
    return np.mean(errors_cv, axis = 0)

In [ ]:
"""Kernel approximation method selection"""
def approximation(X, kernel, n):
    if kernel == 'random kitchen sinks':
        X_kernel = random_kitchen_sink(X, n)
    elif kernel == 'FastFood':
        m, d = X.shape[0], X.shape[1]
        X_kernel = FF.fastfood_forkernel(X, n)
        X_kernel = X_kernel[:,0:n]
    return X_kernel

# Adult Data Set

In [ ]:
adult = pd.read_csv('adult_data.csv',sep = ';')
n = adult.shape[0]
X = adult[adult.columns[0:adult.shape[1]-1]]
y = adult[adult.columns[adult.shape[1]-1:adult.shape[1]]]
y = np.reshape(np.array(y),(n, 1))
X, y = shuffle(X, y)

cols = X.columns
X = X.iloc[:10000]
y = y[:10000]
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns = cols)
# X.iloc[:,:] = preprocessing.Normalizer(norm='l2').fit_transform(X)
X = X.values
# y = preprocessing.normalize(y, norm='l2')
normX = np.max(np.linalg.norm(X, axis=1))
X = X/normX

In [ ]:
print(X.shape, y.shape)

In [ ]:
print('Label rate is ' + str(np.sum(y==1)/10000))

# KDD Data Set

In [ ]:
kddcup99 = pd.read_csv('kddcup99.csv', sep = ';')
n,d = kddcup99.shape
y = np.reshape(np.array(kddcup99['label']),(n, 1))
X = kddcup99.drop(columns = ['label'])
X, y = shuffle(X, y, random_state = 0)

X = X.iloc[:15000]
y = y[:15000]

cols = X.columns

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns = cols)
X.iloc[:,:] = preprocessing.Normalizer(norm='l2').fit_transform(X)
X = X.values

y = preprocessing.normalize(y, norm='l2')
# X, y = shuffle(X, y, random_state = 0)



print(X.shape, y.shape)
print('Label rate is ' + str(np.sum(y==1)/15000))

# Original Nonprivate 

In [ ]:
clf = svm.SVC(gamma='scale')
scores = cross_val_score(clf, X, y.flatten(), cv=10)
errors_svm = np.mean([1 - score for score in scores])
errors_svm

# Fast Food

In [ ]:
m, d = X.shape
n = d
kernel = 'FastFood'
epsilons = [0.1]

## Select best Lambda

In [ ]:
non_pri = 'non_private'
print(non_pri)
print('##############')
Lambda = [10**(-10), 10**(-7), 10**(-4), 10**(-3.5), 10**(-3), 10**(-2.5), 10**(-2), 10**(-1.5)]
t0 = time.time()
error_nonpriv_ff_cv, error_nonpriv_ff_svm, beta_opt_nonpriv = trainNonPrivateModel(X, y, non_pri, Lambda, kernel, n)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

In [ ]:
obj = 'objective_pert'
print(obj)
print('##############')

Lambda = [10**(-10), 10**(-7), 10**(-4), 10**(-3.5), 10**(-3), 10**(-2.5), 10**(-2), 10**(-1.5)]
t0 = time.time()
error_ff_obj_best_lambda, beta_opt_obj = trainAggregateModel(X, y, obj, kernel, Lambda, kernel, n, epsilons)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

In [ ]:
out = 'output_pert'
print(out)
print('##############')
Lambda = [10**(-10), 10**(-7), 10**(-4), 10**(-3.5), 10**(-3), 10**(-2.5), 10**(-2), 10**(-1.5)]
errors = []
t0 = time.time()
error_ff_out_best_lambda, beta_opt_out = trainAggregateModel(X, y, obj, kernel, Lambda, kernel, n, epsilons)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

## Use fixed Lambda

In [ ]:
epsilons = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

In [ ]:
obj = 'objective_pert'
print(obj)
print('##############')

Lambda = [0.00031622776601683794]
t0 = time.time()
error_ff_obj_fixed_lambda, beta_opt_obj = trainAggregateModel(X, y, obj, kernel, Lambda, kernel, n, epsilons)
t1 = time.time()
print('Runtime : ' + str(t1-t0))


In [ ]:
out = 'output_pert'
print(out)
print('##############')
Lambda = [0.0031622776601683794]
t0 = time.time()
error_ff_out_fixed_lambda, beta_opt_out = trainAggregateModel(X, y, out, kernel, Lambda, kernel, n, epsilons)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

# Random Kitchen Sinks

In [ ]:
kernel = 'random kitchen sinks'
m, d = X.shape
n = d

## Select best Lambda

In [ ]:
epsilons = [0.1]

In [ ]:
non_pri = 'non_private'
print(non_pri)
print('##############')
Lambda = [10**(-10), 10**(-7), 10**(-4), 10**(-3.5), 10**(-3), 10**(-2.5), 10**(-2), 10**(-1.5)]
t0 = time.time()
error_nonpriv_rks_cv, error_nonpriv_rks_svm, beta_opt_nonpriv_rks = trainNonPrivateModel(X, y, non_pri, Lambda, kernel, n)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

In [ ]:
obj = 'objective_pert'
print(obj)
print('##############')
Lambda = [10**(-10), 10**(-7), 10**(-4), 10**(-3.5), 10**(-3), 10**(-2.5), 10**(-2), 10**(-1.5)]
t0 = time.time()
error_rks_obj_best_lambda, beta_obj_nonpriv_rks = trainAggregateModel(X, y, obj, kernel, Lambda, kernel, n, epsilons)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

In [ ]:
out = 'output_pert'
print(out)
print('##############')
Lambda = [10**(-10), 10**(-7), 10**(-4), 10**(-3.5), 10**(-3), 10**(-2.5), 10**(-2), 10**(-1.5)]
t0 = time.time()
error_rks_out_best_lambda, beta_out_nonpriv_rks = trainAggregateModel(X, y, out, kernel, Lambda, kernel, n, epsilons)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

## Use fixed Lambda

In [ ]:
epsilons = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

In [ ]:
obj = 'objective_pert'
print(obj)
print('##############')
Lambda = [0.0001]
t0 = time.time()
error_rks_obj_fixed_lambda = trainAggregateModel(X, y, obj, kernel, Lambda, kernel, n, epsilons)
t1 = time.time()
print('Runtime : ' + str(t1-t0))

In [ ]:
out = 'output_pert'
print(out)
print('##############')
Lambda = [0.01]
t0 = time.time()
error_rks_out_fixed_lambda = trainAggregateModel(X, y, out, kernel, Lambda, kernel, n, epsilons)   
t1 = time.time()
print('Runtime : ' + str(t1-t0))

# Plot Best Lambda

In [ ]:
epsilon = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
error_ff = [error_nonpriv_ff_cv] * 11

plt.plot(epsilon, error_ff)
plt.plot(epsilon, error_ff_obj_fixed_lambda)
plt.plot(epsilon, error_ff_out_fixed_lambda)
# plt.ylim(0.15, 0.45)
my_xticks = ['0.01', '0.05', '0.1', '0.15', '0.2', '0.25', '0.3', '0.35', '0.4', '0.45', '0.5']
plt.xticks(epsilon, my_xticks)
plt.xlabel('Private Parameter')
plt.ylabel('Misclassification Error Rate')
plt.legend(['NonPrivate','Objective Perturbation','Output Perturbation'], loc='upper center')
plt.savefig('error_rate_FastFood_best_Lambda.png')
plt.show()

In [ ]:
epsilon = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
error_rks = [error_nonpriv_rks_cv] * 11
plt.plot(epsilon, error_rks)
plt.plot(epsilon, error_rks_obj_fixed_lambda)
plt.plot(epsilon, error_rks_out_fixed_lambda)
# plt.ylim(0, 0.1)
my_xticks = ['0.01', '0.05', '0.1', '0.15', '0.2', '0.25', '0.3', '0.35', '0.4', '0.45', '0.5']
plt.xticks(epsilon, my_xticks)
plt.xlabel('Private Parameter')
plt.ylabel('Misclassification Error Rate')
plt.legend(['NonPrivate','Objective Perturbation','Output Perturbation'], loc='upper center')
plt.savefig('error_rate_RKS_best_Lambda.png')
plt.show()